# Diagnoses Meta-Analysis Pediatrics

This notebook contain the R code to conduct meta-analysis in terms of 
- percentage of patients with each ICD code
- the 95% confidence interval of the percentage of patients with each ICD code

The meta-analysis results in this script are based on the site level results. Before running this script, please run DiagnosisSummary.ipynb first, and save the site level results "siteSubsetOutput.txt" to your working directory.

In [ ]:
rm(list=ls())
# Set working directory where the file "siteSubsetOutput.txt" are
setwd("./4CE/phase1.1/")

#############
# LIBRARIES #
#############
library(metafor)
library(data.table)

###################
# LOAD INPUT DATA #
###################
dat=data.frame(fread("siteSubsetOutput.txt"))

Then we create a function for random-effects meta-analysis.

In [ ]:
meta.FUN=function(code,dat){
  dat.sub=dat[dat$icd==code,c("description","num_patients_all_since_admission_diagnosis","totalPatients")]
  dat.sub[dat.sub[,2]==0,c(2,3)]=dat.sub[dat.sub[,2]==0,c(2,3)]+0.5
  yi=dat.sub[,"num_patients_all_since_admission_diagnosis"]/dat.sub[,"totalPatients"]
  vi=yi*(1-yi)/dat.sub[,"totalPatients"]
  junk=rma(yi=yi, vi=vi, method="DL")
  res=c(junk$beta, junk$ci.lb, junk$ci.ub)
  res=c(description=dat.sub$description[1], code=code, res)
  res
  }

We conduct meta-analysis.

In [ ]:
code.list=unique(dat$icd)
res_Meta=data.frame(do.call(rbind, lapply(code.list, function(code) meta.FUN(code,dat))))
rownames(res_Meta)=NULL